## 1. Load Dataset and Fit Data for Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load CSV file
df = pd.read_csv('your_dataset.csv')  # Replace 'your_dataset.csv' with dataset name

# Separate features and target variable
X = df.drop('Label_Column', axis=1)   # Replace 'Label_Column' with target column
y = df['Label_Column']               # Replace 'Label_Column' with target column

# Check the first few rows of the data
print(df.head())

# Split original dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Testdata: 0.2 - Traindata: 0.6 (0.8*0.75) - Validationdata: 0.2 (0.8*0.25)
XX_train, X_val, yy_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

## 2. Split original dataset into Three:
- Training Dataset
- Validation Dataset (Testdataset for Hyperparameter Optimization process)
- Test Dataset (for final Evaluation)

## (Simple Test Dataset to Check Code - Should be Deleted Later)

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# Lade den Iris-Datensatz
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='target')
# Entferne alle Einträge mit dem Label 2
X = X[y != 2]
y = y[y != 2]
# Prüfen der neuen Labels
print("Einzigartige Labels nach Umbenennung:", y.unique())
# Split original dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Testdata: 0.2 - Traindata: 0.6 (0.8*0.75) - Validationdata: 0.2 (0.8*0.25)
XX_train, X_val, yy_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

Einzigartige Labels nach Umbenennung: [0 1]


## 3. Perform Hyperband Hyperparameter-Optimization

In [2]:
# Because optuna library gets deactivated all the time within this environment
try:
    import optuna
    print("Optuna ist bereits installiert.")
except ImportError:
    import os
    os.system('pip install optuna')

Optuna ist bereits installiert.


In [3]:
import optuna
from optuna.pruners import HyperbandPruner
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Function for Hyperparameter-Optimization
def objective(trial):
    # Hyperparameter search space
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 1.0)
    
    # AdaBoost-Model code with full parameter search space
    model = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    model.fit(XX_train, yy_train)
    
    # Accuracy on test-set
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

# Activate Hyperband-Pruner
study = optuna.create_study(direction="maximize", pruner=HyperbandPruner())

# objective: value to optimize (maximize or minimize e.g. direction="maximize" like defined above)
# n_trials: How many Hyperparameter combinations will be tried
study.optimize(objective, n_trials=50)

[I 2025-01-12 14:45:04,170] A new study created in memory with name: no-name-f09b6422-45cd-43a4-8318-3a53f1999083
/tmp/ipykernel_831/200807207.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 1.0)
[I 2025-01-12 14:45:04,178] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 99, 'learning_rate': 0.1688926781647169}. Best is trial 0 with value: 1.0.
/tmp/ipykernel_831/200807207.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 1.0)
[I 2025-01-12 14:45:04,185] Trial 1 finished with value: 1.0 and parameters: {'n_estimators':

In [4]:
# Print best parameter and test results
print("Best Parameter: ---", study.best_params)
print("Best Accuracy: ----", study.best_value)

Best Parameter: --- {'n_estimators': 99, 'learning_rate': 0.1688926781647169}
Best Accuracy: ---- 1.0


## 4. Train Model With Best Parameters

In [5]:
# Create model with best hyperparameter
best_ada_model = AdaBoostClassifier(**study.best_params, random_state=42)

# Train final model with training- and validation dataset combined
best_ada_model.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=0.1688926781647169, n_estimators=99,
                   random_state=42)

## 5. Evaluate Final Model

In [6]:
# Predict test data
y_pred = best_ada_model.predict(X_test)  

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# Show results
print(f"Model: {best_ada_model.__class__.__name__}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Combined Confusion Matrix:")
print(cm)
print("-" * 40)

Model: AdaBoostClassifier
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
Combined Confusion Matrix:
[[12  0]
 [ 0  8]]
----------------------------------------


## 6. Resignations-Probability Outputs

In [7]:
# Same formula like above with probability ouputs
y_proba = best_ada_model.predict_proba(X_test)

#Probability that prediciton is 1 (Resignation/Austritt)
y_proba[:,1]

array([0.88079708, 0.88079708, 0.88079708, 0.11920292, 0.11920292,
       0.11920292, 0.11920292, 0.88079708, 0.11920292, 0.11920292,
       0.11920292, 0.11920292, 0.88079708, 0.11920292, 0.88079708,
       0.11920292, 0.88079708, 0.88079708, 0.11920292, 0.11920292])